# Installing and Loading Packages

In [0]:
!pip install tensorflow-gpu==2.0.0

In [0]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print("Tensorflow Version: {}".format(tf.__version__))
print("Device List: {}".format(device_lib.list_local_devices()))

# Quickstarts for Beginners

## Data Preparation

In [0]:
mnist = tf.keras.datasets.mnist

In [0]:
(x_train, t_train), (x_test, t_test) = mnist.load_data()

In [0]:
print(x_train.shape, t_test.shape)

### Data Preprocessing

Normalize the image into range from 0 to 1.

In [0]:
x_train, x_test = x_train / 255.0, x_test / 255.0

## Building Models

In [0]:
model_seq = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [0]:
model_seq.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

## Train and Evaluate Models

In [0]:
model_seq.fit(x_train, t_train, epochs=10)

In [0]:
model_seq.evaluate(x_test, t_test, verbose=2)

# Quickstarts for experts

## Data Preparation

In [0]:
mnist = tf.keras.datasets.mnist

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### Data Preprocessing

In [0]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [0]:
print(x_train.shape, y_train.shape)

Expand a new axis to the x_train and x_test. Here we are going to build a convolution neural network.

In [0]:
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
print(x_train.shape, x_test.shape)

Use `tf.data` to batch and shuffle the dataset.

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Building Models

Build the `tf.keras` model via Keras model APIs.

In [0]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
    self.flatten = tf.keras.layers.Flatten()
    self.d1 = tf.keras.layers.Dense(128, activation='relu')
    self.d2 = tf.keras.layers.Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

Instantiate a model object.

In [0]:
model = MyModel()

You can build a model via a more forward way.

In [0]:
def CNNModel(inputs):
  x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding='same')(inputs)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dense(10, activation='softmax', name="output")(x)
  return x

In [0]:
inputs = tf.keras.Input(shape=(28, 28, 1), dtype=tf.float32)
outputs = CNNModel(inputs)
model = tf.keras.Model(inputs, outputs)
model.summary()

## Train and evaluate the model.

Define a loss function and an optimizer.

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

In [0]:
optimizer = tf.keras.optimizers.Adam()

Define metrics to measure the loss and accuracy of a model.

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [0]:
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Use `tf.GradientTape` to train the model.

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

Test the model.

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
EPOCHS = 10

In [0]:
for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))